## About WindTrace
WindTrace is a python open-source parametric life-cycle inventory model to create tailor-made inventories of onshore wind turbines and park fleets and assess their environmental impacts. 

**IMPORTANT**: in order to run WindTrace you must have an Ecoinvent license. WindTrace is currently compatible with Ecoinvent 3.9.1, 3.10 and 3.10.1. 

More information on GitHub: https://github.com/LIVENlab/WindTrace_public

Copyright (c) 2025 LIVEN lab 
Licensed under the MIT License. See LICENSE for details.
This work has been funded by the European Commission though the Horizon Europe project "JUSTWIND4ALL" (GA 101083936). 

## WindTrace step-by-step

### Outlook
#### 1. Create environment and install the requirements
#### 2. Set consts.py variables
#### 3. Create/open a brightway project
#### 4. Creating life-cycle inventories
##### 4.1 Example of application
##### 4.2 Exploring the inventories
##### 4.3 Calculating LCIA scores
##### 4.4 Another example with land use focus
##### 4.5 Creating a future wind park

### 1. Create environment and install the requirements

Open the conda console and run the following line. Be sure you are in the folder where the environment.yml file is saved.
```ruby
conda env create -f environment.yml 
```
This will create a new environment called 'windtrace_env' with all the packages you need to run this Jupyter Notebook already installed. \
Make sure to activate this environment before running this Notebook. Be also sure you have anaconda or miniconda installed in your computer. 

### 2. Set consts.py variables

**IMPORTANT**: Before running anything, in the file consts.py set the following variables: PROJECT_NAME, SPOLD_FILES, and NEW_DB_NAME.
- PROJECT_NAME: name of the brightway project to be used.
- SPOLD_FILES: folder path where you have the Spoldfiles from Ecoinvent in your local disk. ONLY IF YOU DO NOT USE THE ecoinvent_interface PACKAGE
- NEW_DB_NAME: name of the databes where all the new wind turbine inventories you create will be stored.


### 3. Create/open a brightway project
You first need to import the necessary packages:

In [ ]:
import bw2io as bi
from WindTrace_onshore import *

Then the first step is to set the Brightway project where you will work. 

In [ ]:
bd.projects.set_current(consts.PROJECT_NAME)
bi.bw2setup()

Once you have set your BW project, you can import the Ecoinvent database you need (if it is not already present).
You may do this with one of these two options: 
<br> 
#### Option 1: Adding ecoinvent from local directory
If you are using this option, then you need to provide the path to the folder where you have the spold files in the consts.py file.

In [ ]:
spold_files = consts.SPOLD_FILES
if "cutoff391" not in bd.databases:
    ei = bi.SingleOutputEcospold2Importer(spold_files, "cutoff391", use_mp=False)
    ei.apply_strategies()
    ei.write_database()
if consts.NEW_DB_NAME not in bd.databases:
    new_db = bd.Database(consts.NEW_DB_NAME)
    new_db.register()

#### Option 2. Adding ecoinvent with the Ecoinvent interface
Alternatively, you can also use ecoinvent_interface to download and install the Ecoinvent databases that you wish among versions 3.9.1, 3.10 and 3.10.1.

You need to provide your username and password to access ecoinvent

In [ ]:
from ecoinvent_interface import Settings, permanent_setting, EcoinventRelease, ReleaseType
import os

bd.projects.set_current(consts.PROJECT_NAME)

SYSTEM_MODEL = "cutoff"
# This will install the biosphere and technosphere for Ecoinvent versions 3.9.1, 3.10 and 3.10.1. In case you 
# only want to work with one of the versions, eliminate the others from the list
EI_RELEASE_VERSIONS = ["3.9.1", "3.10", "3.10.1"]
FILE_REPOSITORY = r'YOUR_FILE_PATH'  # add the path where you would like to save the Ecoinvent databases

permanent_setting("username", "YOUR_USERNAME")  # change for your Ecoinvent username
permanent_setting("password", "YOUR_PASSWORD")  #change for your Ecoinvent password
permanent_setting("output_path", FILE_REPOSITORY)


# this will download the Ecoinvent database/s for the versions you specified and write them in your brightway25 project
my_settings = Settings()
release = EcoinventRelease(my_settings)

for release_version in EI_RELEASE_VERSIONS:
    eidb_name = f'eidb_{release_version}'
    biodb_name = f"biosphere_{release_version}"
    release_dir = os.path.join(FILE_REPOSITORY, f'{release_version}_ecoSpold02')
    if not os.path.exists(release_dir):
        release.get_release(release_version, SYSTEM_MODEL, ReleaseType.ecospold)
    else:
        print(f"Release already exists at {release_dir}")
    try:
        bi.import_ecoinvent_release(release_version, SYSTEM_MODEL, use_mp=False)
        print(f"Ecoinvent {release_version} ({SYSTEM_MODEL}) imported successfully!")
    except Exception as e:
        print(f"Failed to import ecoinvent release: {e}")

Now the database where the turbine inventories will be installed (new_db), the biosphere database (biosphere) and the ecoinvent database (ei_cutoff) must be initialised. \
**IMPORTANT**: note that the name of the database will change depending if you used the conventional bi.bw2setup() or ecoinvent_interface to install your databases.

In [ ]:
new_db = bd.Database(consts.NEW_DB_NAME)
biosphere = bd.Database('biosphere3')  # change for "ecoinvent-3.10-biosphere" if you are using ecoinvent 3.10 from ecoinvent_interface
ei_cutoff = bd.Database("cutoff391")  # change for "ecoinvent-3.10-cutoff" if you are using ecoinvent 3.10 from ecoinvent_interface

### 4. Creating life-cycle inventories

The function allowing to create customized inventories is **lci_wind_turbine()**, which admits the following parameters:
| Parameter                     | Description                                                                  | Default                             | Type |
|------------------------------|------------------------------------------------------------------------------|-------------------------------------|------|
| Park name                    | Name of the wind park                                                        | *Required*                          | `str` |
| Park power                   | Total power of the wind park (MW)                                            | *Required*                          | `float` |
| Number of turbines           | Number of turbines in the wind park                                          | *Required*                          | `int` |
| Park location                | Country code (ISO 3166-1 alpha-2)                                            | *Required*                          | `str` |
| Park coordinates             | Latitude and longitude (WGS84)                                               | *Required*                          | `tuple` |
| Manufacturer                 | Turbine manufacturer                                                         | `LM Wind`                           | `'LM Wind'`, `'Vestas'`, `'Siemens Gamesa'`, `'Enercon'`, `'Nordex'` |
| Rotor diameter               | Rotor diameter (m)                                                           | *Required*                          | `int` |
| Turbine rated power          | Nominal power per turbine (MW)                                               | *Required*                          | `float` |
| Hub height                   | Turbine hub height (m)                                                        | *Required*                          | `float` |
| Regression adjustment        | Steel mass as function of `D2h` or `Hub height`                              | `D2h`                               | `'D2h'`, `'Hub height'` |
| Commissioning year           | Year turbine started operation                                               | *Required*                          | `int` |
| Recycled steel share         | Recycled content share of steel                                              | Eurofer data (2012–2021)            | `float` |
| Land use permanent intensity | Permanent land transformation per MW (m²/MW)                                 | `3000`                              | `float` |
| Electricity mix steel        | Electricity mix for steel production                                         | Eurofer country mix                 | `'Norway'`, `'Europe'`, `'Poland'` *(optional)* |
| Generator type               | Gearbox and generator type                                                   | `Doubly fed induction generator`    | `'dd_eesg'`, `'dd_pmsg'`, `'gb_pmsg'`, `'gb_dfig'` *(optional)* |
| Land cover type              | Land cover before installation                                               | *Required*                          | `'crops, non-irrigated'`, `'grassland'`, `'road'`, etc. |
| End-of-life scenario         | Material recycling scenario: <br> `1`: baseline<br> `2`: optimistic<br> `3`: pessimistic | `Baseline` | `str` |
| Lifetime                     | Expected lifetime of the turbine (years)                                     | `20`                                | `int` |
| Capacity factor              | Avg. delivered electricity / theoretical max                                 | `0.24`                              | `float` |
| Attrition rate               | Annual performance degradation                                               | `0.009`                             | `float` |


Parameters without a default value must be filled. Parameters with a default value are not compulsory but can be adapted to the user needs.

### 4.1. Example of application

Let's build an imaginary 100 MW wind park ('my_wind_park') of 20 turbines, 5 MW each, in a random location in Spain. \
The manufacturer of the turbine is Siemens Gamesa, with turbines of 130 m rotor diameter, 140 m hub height and a gear-boxed double-fed induction generation drivetrain. \
The park is commissioned in 2020 and is expected to last for 25 years. \
The expected initial capacity factor is 24%. \
**IMPORTANT**: make sure the name you give to the park (park_name variable) has not been used previously in your database! This means, you cannot create two wind parks with the exact same name in the same database.

In [ ]:
lci_wind_turbine(park_name='my_wind_park', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',
                 lifetime=25,  
                 cf=0.24,
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

If you need to delete the new database: 
WARNING: All inventories created will be deleted!!

In [ ]:
#delete_new_db() # Uncomment this line to delete the new database

### 4.2 Exploring the inventories

The inventories are given by unit of turbine, unit of park, and kWh (for both turbine and park). In this example, the inventoris would have the following names (and codes):

- Turbine (FU=unit): my_wind_park_single_turbine
- Turbine (FU=kWh): my_wind_park_turbine_kwh
- Park (FU=unit): my_wind_park_10.0
- Park (FU=kWh): my_wind_park_kwh

You can access them as follows:

In [ ]:
# park (FU=unit)
park_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_100.0')
park_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_park_kwh')
turbine_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_single_turbine')
turbine_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_turbine_kwh')

Altrnatively, if your interest is only the materials of one turbine, you can access them with the following code (and name):
- Turbine materials: my_wind_park_materials

In [ ]:
turbine_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_materials')
for ex in turbine_act.technosphere():
    print(f'{ex.input["name"]}: {ex["amount"]} ({ex.unit})')
# Note that steel and chromium steel appear both twice. The first appearance corresponds to the turbine, and the second one to the foundations.

### 4.3 Calculating LCIA scores

LCA impacts of a park or an individual turbine of the park can be calculated with lca_wind_turbine(). It returns two dictionaries: the first one being results per unit and the second one being results per kWh. Default results are given per turbine. For the whole park, you must set the parameter 'turbine' to False. ReCiPe 2016 v1.03, midpoint (H) is the default LCIA method. It can be manually changed by giving the parameter 'method' another name of a method in Brightway.

Here an example of application for the whole park. The first dictionary saves scores per FU=unit, and the second per FU=1kWh

In [ ]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', turbine=False, new_db=new_db)
print(f'LCIA scores for (FU = one wind park unit) {lcia_scores_unit}')
print(f'LCIA scores for (FU = one kwh) {lcia_scores_kwh}')

Alternatively, you can give it a certain set of indicators of your choice as a list with the variable 'indicators'. Example:

In [ ]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT'), 
                                           ('Cumulative Energy Demand (CED)', 'energy resources: renewable', 'energy content (HHV)'),
                                           ('Ecological Scarcity 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
                                          ], 
                               turbine=False, new_db=new_db)
print(f'LCIA scores for (FU = one wind park unit) {lcia_scores_unit}')
print(f'LCIA scores for (FU = one kwh) {lcia_scores_kwh}')

### 4.4 Another example with land use focus

Now let's build another wind park changing a bit the land use related parameters. Let's say we know the original land where the turbine will be built is a pasture. Let's say we also know that the permanent land use intensity is smaller than the default number we use. This parameter should include the squared meters per MW of land that will be occupied by permanent access roads, transformer, substation and the turbines themselves. We will use an intensity of 1000 m2/MW this time.

In [ ]:
lci_wind_turbine(park_name='my_wind_park_LandUse', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',  
                 lifetime=25,  
                 cf=0.24,
                 land_use_permanent_intensity=1000,
                 land_cover_type='pasture',
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

And compare the land-use impacts with the previous wind park we created:

In [ ]:
previous_park_score_unit, previous_park_score_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
new_park_score_unit, new_park_score_kwh  = lca_wind_turbine(park_name='my_wind_park_LandUse', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
print(f'Land Use score previous park (FU=one park): {previous_park_score_unit}; (FU=1kWh): {previous_park_score_kwh}')
print(f'Land Use score new park(FU=one park): {new_park_score_unit}; (FU=1kWh): {new_park_score_kwh}')

### 4.5 Creating a future wind park

There are some input variables that allow to model more 'futuristic' wind turbine designs. For instance, if the turbine you are designing is going to be installed 10 years from now, you may want to consider that the steel industry (with a huge impact in wind turbines) has improved its recycling shares, and is using a cleaner electricity mix. Moreover, you can adapt the end-of-life scenario to consider optimistic recycling rates. Finally, you may want to add a higher capacity factor (due to, for instance, technichal improvements). A summary of how would this be implemented:
- Steel recycling rate improvement: recycled_share_steel=0.6 (European recycled steel share in 2021 was 0.435)
- Steel electricity mix improvement: electricity_mix_steel='Norway' (only options: 'Norway', 'Europe', 'Poland')
- Optimistic recycling rates (rr): eol_scenario=2 (eol_scenario=1 means current rr, eol_scenario=2 means optimistic rr, eol_scenario=3 means pessimistic rr)
- Higher capacity factor: cf=0.4 (This is a 40%. The European mean in 2024 was 24%)

In [ ]:
lci_wind_turbine(park_name='my_wind_park_future',
                 park_power=100.0,
                 number_of_turbines=20,
                 park_location='ES',
                 park_coordinates=(41.502, -1.126),
                 manufacturer='Siemens Gamesa',
                 rotor_diameter=130,
                 turbine_power=5.0,
                 hub_height=140,
                 commissioning_year=2030,
                 generator_type='gb_dfig',
                 lifetime=25,
                 cf=0.40,
                 eol_scenario=2,
                 electricity_mix_steel='Norway',
                 recycled_share_steel=0.6,
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

And compare climate change impacts with the previous turbine modelled in section 4.1 in this notebook:

In [ ]:
previous_park_score_unit, previous_park_score_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1',
                               indicators=[('EF v3.1', 'climate change', 'global warming potential (GWP100)')
                                          ],
                               turbine=False, new_db=new_db)
new_park_score_unit, new_park_score_kwh  = lca_wind_turbine(park_name='my_wind_park_future', park_power=100.0, method='EF v3.1',
                               indicators=[('EF v3.1', 'climate change', 'global warming potential (GWP100)')
                                          ],
                               turbine=False, new_db=new_db)
print(f'Climate Change score previous park (FU=one park): {previous_park_score_unit}; (FU=1kWh): {previous_park_score_kwh}')
print(f'Climate Change score new park(FU=one park): {new_park_score_unit}; (FU=1kWh): {new_park_score_kwh}')